# Large Actor Experiment - Testing if Larger Architecture Prevents Saturation

**Hypothesis:** Making the actor have the same hidden layer sizes as the critic (512 -> 128 instead of 64 -> 32) may help avoid tanh saturation.

## Architecture Comparison

| Network | Original | Large Actor (This Experiment) |
|---------|----------|-------------------------------|
| Actor   | 7->64->32->3 | 7->**512**->**128**->3 |
| Critic  | 510->512->128->1 | 510->512->128->1 |

## Learning Rate Order (High to Low)
Experiments run from **HIGH to LOW** learning rates to prioritize early-stopping cases:
- Actor LRs: `[0.1, 0.01, 0.001, 0.0001]`
- Critic LRs: `[0.1, 0.01, 0.001, 0.0001]`
- Total: **16 experiments**

---

## Features:
- Auto-saves to Google Drive every 100 episodes
- Auto-restores from Drive on session restart
- Crash recovery built-in
- Early stopping when all actors stop updating

## Cell 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted!")

## Cell 2: Upload and Extract Experiment Files

**Option A:** Upload `large_actor_experiment.zip` when prompted

**Option B:** If you already uploaded to Drive, skip the upload prompt

In [ ]:
import os
import zipfile

# Check if files already exist
if os.path.exists('/content/large_actor_experiment/mec_env.py'):
    print("Experiment files already present!")
else:
    # Try to find zip in Drive first
    drive_zip = '/content/drive/MyDrive/large_actor_experiment.zip'
    
    if os.path.exists(drive_zip):
        print(f"Found zip in Drive: {drive_zip}")
        zip_path = drive_zip
    else:
        # Upload zip file
        print("Upload large_actor_experiment.zip:")
        from google.colab import files
        uploaded = files.upload()
        zip_path = list(uploaded.keys())[0]
    
    # Extract
    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall('/content')
    
    print("\nExtracted files:")
    for item in os.listdir('/content'):
        if not item.startswith('.'):
            print(f"  {item}")

## Cell 3: Check GPU and Environment

In [ ]:
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("")
    print("WARNING: No GPU detected!")
    print("Go to: Runtime -> Change runtime type -> GPU")

# Set working directory
os.chdir('/content')
print(f"\nWorking directory: {os.getcwd()}")

## Cell 4: Verify Large Actor Architecture

In [ ]:
import sys
sys.path.insert(0, '/content/large_actor_experiment')

from Model import LargeActorNetwork, ActorNetwork, CriticNetwork
import torch

# Compare architectures
small_actor = ActorNetwork(7, 3, torch.tanh)
large_actor = LargeActorNetwork(7, 3, torch.tanh)
critic = CriticNetwork(350, 150, 7, 3)  # 50 agents * 7 state, 50 agents * 3 action

print("=" * 50)
print("ARCHITECTURE COMPARISON")
print("=" * 50)
print(f"\nSmall Actor (original): 7 -> 64 -> 32 -> 3")
print(f"  Parameters: {sum(p.numel() for p in small_actor.parameters()):,}")
print(f"\nLarge Actor (this experiment): 7 -> 512 -> 128 -> 3")
print(f"  Parameters: {sum(p.numel() for p in large_actor.parameters()):,}")
print(f"\nCritic: 510 -> 512 -> 128 -> 1")
print(f"  Parameters: {sum(p.numel() for p in critic.parameters()):,}")
print(f"\nLarge Actor / Critic ratio: {sum(p.numel() for p in large_actor.parameters()) / sum(p.numel() for p in critic.parameters()):.2%}")

## Cell 5: Run All 16 Experiments

This will run all 16 experiments with **HIGH learning rates FIRST**:
- Actor LRs: `[0.1, 0.01, 0.001, 0.0001]` (high to low)
- Critic LRs: `[0.1, 0.01, 0.001, 0.0001]` (high to low)

**Progress is auto-saved to Google Drive every 100 episodes.**

In [ ]:
import sys
sys.path.insert(0, '/content')
sys.path.insert(0, '/content/large_actor_experiment')

from run_large_actor_experiment import run_all_experiments

# Run all experiments
run_all_experiments()

## Cell 6: Check Experiment Status

In [ ]:
import json
import os

results_dir = '/content/results/large_actor_experiment'
status_file = os.path.join(results_dir, 'experiment_status.json')

if os.path.exists(status_file):
    with open(status_file) as f:
        status = json.load(f)
    print("Experiment Status:")
    print(f"  Completed: {len(status['completed'])}/16")
    if status['in_progress']:
        print(f"  In progress: {status['in_progress']}")
    print("\nCompleted experiments:")
    for exp in sorted(status['completed']):
        print(f"  - {exp}")
else:
    print("No status file found yet.")

# Also check Drive backup
drive_dir = '/content/drive/MyDrive/large_actor_results'
if os.path.exists(drive_dir):
    print(f"\nDrive backup exists: {drive_dir}")
    contents = os.listdir(drive_dir)
    print(f"  Contents: {contents[:5]}..." if len(contents) > 5 else f"  Contents: {contents}")

## Cell 7: View Results Summary

In [ ]:
import json
import os

results_dir = '/content/results/large_actor_experiment'

if os.path.exists(results_dir):
    print("Large Actor Experiment Results Summary:")
    print("="*70)
    print(f"{'Actor LR':<12} {'Critic LR':<12} {'Stop Ep.':<12} {'Final Reward':<15}")
    print("-"*70)
    
    results = []
    for exp_dir in sorted(os.listdir(results_dir)):
        result_file = os.path.join(results_dir, exp_dir, 'results.json')
        if os.path.exists(result_file):
            with open(result_file) as f:
                data = json.load(f)
            results.append(data)
    
    # Sort by actor_lr (descending) then critic_lr (descending)
    for data in sorted(results, key=lambda x: (-x['actor_lr'], -x['critic_lr'])):
        print(f"{data['actor_lr']:<12} {data['critic_lr']:<12} {data['stopping_episode']:<12} {data['final_reward']:<15.4f}")
    print("="*70)
else:
    print("No results directory found yet.")

## Cell 8: Compare with Original Experiment (if available)

In [ ]:
import json
import os

def load_results(results_dir):
    results = []
    if os.path.exists(results_dir):
        for exp_dir in os.listdir(results_dir):
            result_file = os.path.join(results_dir, exp_dir, 'results.json')
            if os.path.exists(result_file):
                with open(result_file) as f:
                    results.append(json.load(f))
    return results

large_results = load_results('/content/results/large_actor_experiment')
original_results = load_results('/content/results/stopping_experiment')

# Also try Drive backups
if not original_results:
    original_results = load_results('/content/drive/MyDrive/gradient_asymmetry_results')

if original_results and large_results:
    print("COMPARISON: Original (Small Actor) vs Large Actor")
    print("="*85)
    print(f"{'Actor LR':<10} {'Critic LR':<10} {'Original Stop':<15} {'Large Stop':<15} {'Difference':<15}")
    print("-"*85)
    
    for orig in sorted(original_results, key=lambda x: (-x['actor_lr'], -x['critic_lr'])):
        large = next((r for r in large_results 
                     if r['actor_lr'] == orig['actor_lr'] and r['critic_lr'] == orig['critic_lr']), None)
        if large:
            diff = large['stopping_episode'] - orig['stopping_episode']
            diff_str = f"+{diff}" if diff > 0 else str(diff)
            improved = "IMPROVED" if diff > 100 else ""
            print(f"{orig['actor_lr']:<10} {orig['critic_lr']:<10} {orig['stopping_episode']:<15} {large['stopping_episode']:<15} {diff_str:<10} {improved}")
    print("="*85)
elif large_results:
    print("Original experiment results not found for comparison.")
    print("Upload original results to '/content/results/stopping_experiment' to compare.")
else:
    print("No large actor results found yet. Run the experiment first.")

## Cell 9: Download Results (Optional)

Results are auto-backed up to Drive, but you can also download a zip.

In [ ]:
import shutil
import os

results_dir = '/content/results/large_actor_experiment'
output_zip = '/content/large_actor_results.zip'

if os.path.exists(results_dir):
    shutil.make_archive('/content/large_actor_results', 'zip', results_dir)
    print(f"Created: {output_zip}")
    print(f"Size: {os.path.getsize(output_zip) / 1024:.1f} KB")
    
    # Download
    from google.colab import files
    files.download(output_zip)
else:
    print("No results directory found.")